In [58]:
# parameters
#file = r'C:\Users\roh\Downloads\1602bbe3-3ba6-4d5f-8f80-79cfe019dbc5.xlsx'
#file = r'C:\Users\roh\Downloads\countrystoreproduct.xlsx'

# excel download from DClick
file = 'C:/Users/roh/Downloads/d5c7a4f7-b9a7-4d1e-904e-ce7349e0f27c.xlsx'

# country parameter
country = ['ALL'] # 'Korea'
category_field1 = 2 #0
category_field2 = 4 #1


In [59]:
import pandas as pd
import numpy as np

# sample of excel download from DClick
df1 = pd.read_excel(open(file, 'rb'), sheet_name='Sheet1')
df1

Country                          Store Retail product L1  \
0      Thailand           Bangkok Siam Paragon    Women Universe   
1         Korea  Seoul Shinsegae Kangnam Women    Women Universe   
2         Korea         Seoul Lotte Main Women    Women Universe   
3         Korea       Seoul Lotte Jamsil Women    Women Universe   
4         Macau           Macau Venetian Women    Women Universe   
...         ...                            ...               ...   
1138  Hong Kong   Hong Kong Times Square Women               PCD   
1139  Australia      Sydney George Street Temp    Women Universe   
1140  Australia             Australia Pop-Up 2    Women Universe   
1141  Hong Kong   Hong Kong Times Square Women      Men Universe   
1142  Australia       Melbourne Collins Street              Baby   

     Retail product L2  Day Net Sales (kEUR) TY  Day Net Sales (kEUR) LY  \
0               W Bags               176.524848               124.058032   
1               W Bags               212.693576               141.442067   
2               W Bags                89.944064                91.147781   
3               W Bags               105.575090               112.100904   
4               W Bags                49.442706               234.154194   
...                ...                      ...                      ...   
1138               PCD                 0.000000                 0.000000   
1139            W Bags                 0.000000                72.296187   
1140     W Accessories                 0.000000                 0.000000   
1141     M Accessories                 0.000000                 0.000000   
1142              Baby                 0.000000                 0.000000   

     Day Net Sales Variation  WTD Net Sales (kEUR) TY  \
0                   0.422922               812.291502   
1                    0.50375               785.393278   
2                  -0.013206               391.301011   
3                  -0.058214               512.145911   
4                  -0.788846               492.301146   
...                      ...                      ...   
1138                       -                 0.000000   
1139                      -1                 0.000000   
1140                       -                 0.000000   
1141                       -                 0.000000   
1142                       -                 0.000000   

      WTD Net Sales (kEUR) LY WTD Net Sales Variation  \
0                  724.713933                0.120844   
1                  432.933636                0.814119   
2                  423.720902               -0.076512   
3                  511.399781                0.001459   
4                 1119.722626               -0.560337   
...                       ...                     ...   
1138                 0.000000                       -   
1139               261.770897                      -1   
1140                 0.000000                       -   
1141                 0.000000                       -   
1142                 0.000000                       -   

      MTD Net Sales (kEUR) TY  MTD Net Sales (kEUR) LY  \
0                  552.733193               463.831670   
1                  607.376931               255.342836   
2                  295.872954               295.449213   
3                  331.502500               313.651603   
4                  331.339018               685.005648   
...                       ...                      ...   
1138                 0.000000                 0.000000   
1139                 0.000000               139.465394   
1140                 0.000000                 0.000000   
1141                 0.000000                 0.000000   
1142                 0.000000                 0.000000   

     MTD Net Sales Variation  YTD Net Sales (kEUR) TY  \
0                   0.191668             43496.831739   
1                   1.378672             31605.343943   
2                   0.001434            

In [60]:
# tablulating subtotals dynamically in pandas table
# https://towardsdatascience.com/tabulating-subtotals-dynamically-in-python-pandas-pivot-tables-6efadbb79be2

def pivot_table_w_subtotals(df, values, indices, columns, aggfunc, fill_value):
    '''
    Adds tabulated subtotals to pandas pivot tables with multiple hierarchical indices.
    
    Args:
    - df - dataframe used in pivot table
    - values - values used to aggregrate
    - indices - ordered list of indices to aggregrate by
    - columns - columns to aggregrate by
    - aggfunc - function used to aggregrate (np.max, np.mean, np.sum, etc)
    - fill_value - value used to in place of empty cells
    
    Returns:
    -flat table with data aggregrated and tabulated
    
    '''
    listOfTable = []
    for indexNumber in range(len(indices)):
        n = indexNumber+1
        table = pd.pivot_table(df,values=values,index=indices[:n],columns=columns,aggfunc=aggfunc,fill_value=fill_value).reset_index()
        for column in indices[n:]:
            table[column] = ''
        listOfTable.append(table)
    concatTable = pd.concat(listOfTable).sort_index()
    concatTable = concatTable.set_index(keys=indices)
    return concatTable.sort_index(axis=0,ascending=True)



In [61]:
# apply pivot table to excel data

df = pd.read_excel(open(file, 'rb'), sheet_name='Sheet1')
if country[0].lower() == 'all':
    new_table = df
else:
    new_table = df[df['Country'].isin(country)]
values = list(df.columns)[4:]  # define list of key facts - from column 5 onwards
indices = list(df.columns)[category_field1:category_field2] # define list of categorisation columns
#columns = list(df.columns)[2:3]
df = pivot_table_w_subtotals(df=new_table, indices=indices, columns=[], values=values, \
          aggfunc='sum',fill_value='')#.sort_values([('Day Net Sales (kEUR) TY')], ascending=False)
#df.sort_values(['Country','YTD Net Sales (kEUR) TY'], ascending=[1,0], inplace=True)
#df.sort_values(['YTD Net Sales (kEUR) TY'], ascending=[0], inplace=True)

#df['equal_or_lower_than_4?'] = df['set_of_numbers'].apply(lambda x: 'True' if x <= 4 else 'False')
#df['Day chg']=df['Day Net Sales (kEUR) LY'].apply(lambda x: df['Day Net Sales (kEUR) TY']/df['Day Net Sales (kEUR) LY']-1
#                                                 if x!=0 else 0)

df['Day chg']=df['Day Net Sales (kEUR) TY']/df['Day Net Sales (kEUR) LY']-1
df['WTD chg']=df['WTD Net Sales (kEUR) TY']/df['WTD Net Sales (kEUR) LY']-1
df['MTD chg']=df['MTD Net Sales (kEUR) TY']/df['MTD Net Sales (kEUR) LY']-1
df['YTD chg']=df['YTD Net Sales (kEUR) TY']/df['YTD Net Sales (kEUR) LY']-1
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df=df[['Day Net Sales (kEUR) TY','Day Net Sales (kEUR) LY','Day chg',
    'WTD Net Sales (kEUR) TY','WTD Net Sales (kEUR) LY','WTD chg',
    'MTD Net Sales (kEUR) TY','MTD Net Sales (kEUR) LY','MTD chg',
    'YTD Net Sales (kEUR) TY','YTD Net Sales (kEUR) LY','YTD chg']]

#df.crosstab()
#df.sort_values(['Retail product L1','YTD Net Sales (kEUR) TY'], ascending=[1,0], inplace=True)
# https://towardsdatascience.com/how-to-do-a-custom-sort-on-pandas-dataframe-ac18e7ea5320
df.sort_values(['Retail product L1','YTD Net Sales (kEUR) TY'], ascending=[0,0], inplace=True)
df

Day Net Sales (kEUR) TY  \
Retail product L1   Retail product L2                            
Women Universe                                     5814.012995   
                    W Bags                         3099.409852   
                    W RTW                           714.140193   
                    W SLG                           612.401644   
                    W Accessories                   747.313120   
                    W Shoes                         640.748187   
                    Haute Couture                     0.000000   
                    W Special                         0.000000   
PCD                                                   0.317644   
                    PCD                               0.317644   
Men Universe                                       1470.198824   
                    M RTW                           685.813411   
                    M Shoes                         253.383398   
                    M Bags                          281.610435   
                    M SLG                           183.459150   
                    M Accessories                    65.932429   
                    M Special                         0.000000   
Jewellery & Watches                                 129.710784   
                    Jewellery                       125.441807   
                    Watches                           4.268977   
Dior Maison                                          22.831765   
                    Maison                           22.831765   
                    Maison Special                    0.000000   
Baby                                                 55.363882   
                    Baby                             55.363882   

                                       Day Net Sales (kEUR) LY   Day chg  \
Retail product L1   Retail product L2                                      
Women Universe                                     5343.377625  0.088078   
                    W Bags                         3019.529347  0.026455   
                    W RTW                           783.389087 -0.088397   
                    W SLG                           498.950279  0.227380   
                    W Accessories                   600.385455  0.244722   
                    W Shoes                         441.123456  0.452537   
                    Haute Couture                     0.000000       NaN   
                    W Special                         0.000000       NaN   
PCD                                                   0.651683 -0.512579   
                    PCD                               0.651683 -0.512579   
Men Universe                                       1170.102960  0.256470   
                    M RTW                           613.646372  0.117604   
                    M Shoes                         179.644796  0.410469   
                    M Bags                          210.411735  0.338378   
                    M SLG                           120.895436  0.517503   
                    M Accessories                    43.631342  0.511125   
                    M Special                         1.873279 -1.000000   
Jewellery & Watches                                 112.563548  0.152334   
                    Jewellery                       109.371339  0.146935   
                    Watches                           3.192210  0.337311   
Dior Maison                                          15.147084  0.507337   
                    Maison                           15.147084  0.507337   
                    Maison Special                    0.000000       NaN   
Baby                                                 98.816436 -0.439730   
                    Baby                             98.816436 -0.439730   

                                       WTD Net Sales (kEUR) TY  \
Retail product L1   Retail product L2                            
Women Universe                                    33941.534227   
     

In [62]:
# apply style sheet / formatting to table

import jinja2  # pip install Jinja2
def color_negative_red(value):
    """
    Colors elements in a dateframe
    green if positive and red if
    negative. Does not color NaN
    values.
    """

    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'

    #return 'color: %s' % color
    return f"color:{color};background-color:lightyellow"

def bold(val, props=''):
    return "font-weight:bold;color:black;background-color:darkgrey;text-align:top"

def highlight_max1(s, props=''):
    return np.where(s != 0.0001, props, '')

def highlight_max(s, props=''):
    return [(props if 'Thailand' in s else None) for s in s.index]

def color_red(val):
    return "color: red"

def color_third_red(value):
    return [('color: red' if 'Third' in x else 'color: black') for x in val.index]

def highlight_x(s):
    return ["vertical-align:text-top;" if "" in v else "" for v in s]
#df.style.apply_index(highlight_x, axis="columns", level=[0, 1])



df_style = (df.style
    .set_caption('Daily Sales Report - ' + ', '.join(country))
    .set_table_styles([
    {'selector': 'caption', 'props': [('font-weight', 'bold'),('color', 'darkblue'),('font-size', '16px')]},
    {'selector': 'th', 'props': [('border-style','solid'),('border-color','lightgrey'),('border-width','0.1px')]}
    ])            
    .applymap(color_negative_red, subset=[col for col in df.columns if 'chg' in col])
    .applymap(bold, subset=slice_)
    .apply_index(highlight_x, axis="index", level=[0])
    .format(precision=0, na_rep='na', thousands=",", 
            formatter={'Day chg': "{:.0%}",
                       'MTD chg': "{:.0%}",
                       'YTD chg': "{:.0%}"
                      })    
)


display(df_style)


In [63]:
# export entire table with style formatting as HTML code for embedding in an email template
print(df_style.to_html())

<style type="text/css">
#T_5df14 caption {
  font-weight: bold;
  color: darkblue;
  font-size: 16px;
}
#T_5df14 th {
  border-style: solid;
  border-color: lightgrey;
  border-width: 0.1px;
}
#T_5df14_row0_col0, #T_5df14_row0_col1, #T_5df14_row0_col3, #T_5df14_row0_col4, #T_5df14_row0_col6, #T_5df14_row0_col7, #T_5df14_row0_col9, #T_5df14_row0_col10, #T_5df14_row8_col0, #T_5df14_row8_col1, #T_5df14_row8_col3, #T_5df14_row8_col4, #T_5df14_row8_col6, #T_5df14_row8_col7, #T_5df14_row8_col9, #T_5df14_row8_col10, #T_5df14_row10_col0, #T_5df14_row10_col1, #T_5df14_row10_col3, #T_5df14_row10_col4, #T_5df14_row10_col6, #T_5df14_row10_col7, #T_5df14_row10_col9, #T_5df14_row10_col10, #T_5df14_row17_col0, #T_5df14_row17_col1, #T_5df14_row17_col3, #T_5df14_row17_col4, #T_5df14_row17_col6, #T_5df14_row17_col7, #T_5df14_row17_col9, #T_5df14_row17_col10, #T_5df14_row20_col0, #T_5df14_row20_col1, #T_5df14_row20_col3, #T_5df14_row20_col4, #T_5df14_row20_col6, #T_5df14_row20_col7, #T_5df14_row20_col9, 